# Predictive Asset Health Analytics

## Introduction
Create an end-to-end predictive asset maintenance solution with XGBoost* from Intel® oneAPI AI Analytics Toolkit (oneAPI). Check out more workflow examples in the [Developer Catalog](https://developer.intel.com/aireferenceimplementations).

## Solution Technical Overview

Predictive asset maintenance is a method that uses data analysis tools to predict defects and anomalies before they happen. Solutions of huge scale typically require operating across multiple hardware architectures. Accelerating training for the ever-increasing size of datasets and machine learning models is a major challenge while adopting AI (Artificial Intelligence).

For an industrial scenario is important to improve the MLOps (Machine Learning Operations) time for developing and deploying new models, this could be challenging due to the ever-increasing size of datasets over a period of time. XGBoost* classifier with HIST tree method addresses this problem improving the overall training/tuning and validation time. A model with a huge set of batch processing requires fast prediction time with a low accuracy lose, daal4py helps the XGBoost* machine learning model to achieve this criteria.

For more details, visit the [Predictive Asset Maintenance](https://github.com/intel-innersource/frameworks.ai.platform.sample-apps.predictive-health-analyticsS) GitHub repository.

## Validated Hardware Details 

Intel® oneAPI is used to achieve quick results even when the data for a model are huge. It provides the capability to reuse the code present in different languages so that the hardware utilization is optimized to provide these results.

| Recommended Hardware           | Precision  |
| ---------------------------- | ---------- |
| Intel® 4th Gen Xeon® Scalable Performance processors|BF16 |
| Intel® 1st, 2nd, 3rd, and 4th Gen Xeon® Scalable Performance processors| FP32 |

## How it Works

This reference kit generates datasets of given row size for a predictive asset maintenance analytics use-case and stores it in ‘. pkl’ format; these data are then split for training and testing, where we train our model built on the XGBoost* algorithm and predict test data.

![Use_case_flow](assets/predictive_asset_maintenance_e2e_flow.png)

## Run Using Jupyter Notebook
### Run Workflow
The following cell provides the variables needed to execute the workflow scripts. 
If the user did not define previously the path to `WORKSPACE` from console or want to use another `WORKSPACE` location replace `<path_defined_by_user>` with the new path. Before using a new `WORKSPACE` directory make sure that the procedure described in `Get Started` described in the `README.md` file has been followed for the new `WORKSPACE` location.

In [ ]:
#Setting path variables.
import os
workspace = os.getenv("WORKSPACE", "<path_defined_by_user>")
data_dir = workspace+'/data'
output_dir = workspace+'/output'
print("workspace path: {}".format(workspace))

#Setting parameter values.
dataset_size = 200000
datapkl_path = data_dir+f"/data_{dataset_size}.pkl"
ncpu = 20
tunning = 0 #Hyperparameter tunning, 0 for no tunning
data_package = "pandas" #Valid options are pandas and modin
cross_validation = 5

The following script can be executed with the parameters provided below for generating the test dataset with the active environment.

```
usage: src/generate_data_pandas.py [-h] [-s SIZE] [-f FILE]

optional arguments:
  -h, --help            show this help message and exit
  -s SIZE, --size SIZE  data size which is number of rows
  -f FILE, --file FILE  output pkl file name
  -d, --debug           changes logging level from INFO to DEBUG
```

In [ ]:
%run {workspace}/src/generate_data_pandas.py -s {dataset_size} -f {datapkl_path}

Training and prediction along with hyperparameter turning can also be executed independently:
```
usage: src/train_predict_pam.py [-h] [-f FILE] [-p PACKAGE] [-t TUNING] [-cv CROSS_VALIDATION] [-patch PATCH_SKLEARN]
                            -ncpu NUM_CPU

optional arguments:
  -h, --help            show this help message and exit
  -f FILE, --file FILE  input pkl file name
  -p PACKAGE, --package PACKAGE
                        data package to be used (pandas, modin)
  -t TUNING, --tuning TUNING
                        hyper parameter tuning (0/1)
  -cv CROSS_VALIDATION, --cross-validation CROSS_VALIDATION
                        cross validation iteration, default 2.
  -ncpu NUM_CPU, --num-cpu NUM_CPU
                        number of cpu cores, default 4.
  -d, --debug           
                        changes logging level from INFO to DEBUG
```

In [ ]:
%run {workspace}/src/train_predict_pam.py -t {tunning} -p {data_package} -f {datapkl_path} -ncpu {ncpu} -cv {cross_validation}

### XGBoost* with oneDAL Python Wrapper (daal4py) model
In order to gain even further improved performance on prediction time for the XGBoost* trained machine learning model, it can be converted to a daal4py model. daal4py makes XGBoost* machine learning algorithm execution faster to gain better performance on the underlying hardware by utilizing the Intel® oneAPI Data Analytics Library (oneDAL).

The generated '.pkl' file is used as input for this Python script. 
```
usage: src/daal_xgb_model.py [-h] [-f FILE]

optional arguments:
  -h, --help            show this help message and exit
  -f FILE, --file FILE  input pkl file name
  -d, --debug           changes logging level from INFO to DEBUG
```
Run the following command to train the model with the given dataset and convert the same to daal4py format and measure the prediction time performance.

In [ ]:
%run {workspace}/src/daal_xgb_model.py -f {datapkl_path}

## Expected Output
A successful execution of ```generate_data_pandas.py``` should return similar results as shown below:
```
INFO:__main__:Generating data with the size 800000
INFO:__main__:changing Tele_Attatched into an object variable
INFO:__main__:Generating our target variable Asset_Label
INFO:__main__:Creating correlation between our variables and our target variable
INFO:__main__:When age is 60-70 and over 95 change Asset_Label to 1
INFO:__main__:When elevation is between 500-1500 change Asset_Label to 1
INFO:__main__:When Manufacturer is A, E, or H change Asset_Label to have  95% 0's
INFO:__main__:When Species is C2 or C5 change Asset_Label to have 90% to 0's
INFO:__main__:When District is NE or W change Asset_Label to have 90% to 0's
INFO:__main__:When District is Untreated change Asset_Label to have 70% to 1's
INFO:__main__:When Age is greater than 90 and Elevaation is less than 1200 and Original_treatment is Oil change Asset_Label to have 90% to 1's
INFO:__main__:=====> Time taken 1.431621 secs for data generation for the size of (800000, 34)
INFO:__main__:Saving the data to /localdisk/aagalleg/frameworks.ai.platform.sample-apps.predictive-health-analytics/data/data_800000.pkl ...
INFO:__main__:DONE
```

A successful execution of ```train_predict_pam.py``` should return similar results as shown below:

```
INFO:__main__:=====> Total Time:
6.791231 secs for data size (800000, 34)
INFO:__main__:=====> Training Time 3.459683 secs
INFO:__main__:=====> Prediction Time 0.281359 secs
INFO:__main__:=====> XGBoost accuracy score 0.921640
INFO:__main__:DONE
```

A successful execution of ```train_predict_pam.py``` should return similar results as shown below:

```
INFO:__main__:Reading the dataset from ./intel_python/data_800000.pkl...
INFO:root:sklearn.model_selection.train_test_split: running accelerated version on CPU
INFO:root:sklearn.model_selection.train_test_split: running accelerated version on CPU
INFO:__main__:XGBoost training time (seconds): 74.001453
INFO:__main__:XGBoost inference time (seconds): 0.054897
INFO:__main__:DAAL conversion time (seconds): 0.366412
INFO:__main__:DAAL inference time (seconds): 0.017998
INFO:__main__:XGBoost errors count: 15622
INFO:__main__:XGBoost accuracy: 0.921890
INFO:__main__:Daal4py errors count: 15622
INFO:__main__:Daal4py accuracy: 0.921890
INFO:__main__:XGBoost Prediction Time: 0.054897
INFO:__main__:daal4py Prediction Time: 0.017998
INFO:__main__:daal4py time improvement relative to XGBoost: 0.672158
INFO:__main__:Accuracy Difference 0.000000
```